In [ ]:
# Init
import os
import sys

module_path = os.path.abspath(os.path.join("../src/simulicronalpha/"))
if module_path not in sys.path:
    sys.path.append(module_path)

# Imports
import random
import numpy as np
import pandas as pd
import warnings
import multiprocessing
np.set_printoptions(suppress=True)
from numpy import concatenate as c

In [ ]:
from generateSim import (generatePopulation, generateGenome)
from stats import stats
from regulation import regulation
from checkCopyNumber import checkCopyNumber

In [ ]:
from popSim import runBatch

In [ ]:
runBatch(    
    numberOfSimulations=1,
    baseSelection=0,
    baseInsertionProb=1,
    numberOfInsertionSites=10000,
    numberOfChromosomes=6,
    baseRecombinationRate=0.01,
    baseTau=1,
    numberOfPiRNA=6,
    piPercentage=3,
    enablePiRecombination=False,
    NumberOfIndividual=10,
    NumberOfTransposonTypes=1,
    NumberOfInsertionsPerType=[1],
    FrequencyOfInsertions=[1.0],
    ExcisionRates=[1.0],
    RepairRates=[1],
    InsertionRates=[1],
    HardyWeinberg=False,
    NumberOfGenerations=10000,
    numberOfThreads=1,
)

In [ ]:
gen,piset,piIndice = generateGenome(
    numberOfInsertionSites=1000,
    numberOfChromosomes=1,
    baseRecombinationRate=0.0,
    baseSelection=0,
)
pop, tr , TEset= generatePopulation(
    gen,
    piIndice,
    NumberOfIndividual=100,
    NumberOfTransposonTypes=1,
    NumberOfInsertionsPerType=[1],
    FrequencyOfInsertions=[1],
    ExcisionRates = [0.1]
)

In [ ]:
insertionSiteFrequencyArray = np.zeros((len(gen), 1,))

In [ ]:
l = stats(pop, tr, TEset, insertionSiteFrequencyArray)

In [ ]:
pop

In [ ]:
unique, counts = np.unique(tr[:,1], return_counts=True)

In [ ]:
transposons = [4,7,11,543,463,323,542,778,342,643]

In [ ]:
tr[transposons,1].astype(int).tolist()

In [ ]:
regulation([1,2,55,1000], TEset, tr, gen)

In [ ]:
pop[1]

In [ ]:
checkCopyNumber(pop)